In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import time
import warnings
import numpy as np
import tensorflow as tf
from sklearn import datasets, linear_model
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
#warnings.filterwarnings("ignore") #Hide messy Numpy warnings

Using TensorFlow backend.


## Load data

In [3]:
def load_data(filename, num_lags, bootstrap_size):
    assert bootstrap_size > num_lags
    
    # read data from file
    f = open(filename)
    series = []
    removed_seasonality = [] 
    removed_std = []
    missings = []
    for line in f:
        splt = line.split(",")
        series.append(float(splt[0]))
        removed_seasonality.append(float(splt[1]))
        removed_std.append(float(splt[2]))
        missings.append(int(splt[3]))
    series = np.array(series)
    removed_seasonality = np.array(removed_seasonality)
    removed_std = np.array(removed_std)
    missings = np.array(missings)
    f.close()

    # generate lags
    X = []
    for i in range(bootstrap_size, len(series)):
        X.append(series[i-num_lags:i])
    X = np.array(X)
    
    y = series[bootstrap_size:]
    removed_seasonality = removed_seasonality[bootstrap_size:]
    removed_std = removed_std[bootstrap_size:]
    missings = missings[bootstrap_size:]
    assert X.shape[0] == y.shape[0]

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, removed_seasonality, removed_std, missings

In [4]:
# read data from different places
NUM_LAGS = 10
bootstrap_size = 12*24*1
ids = []
removed_seasonality_all = []
removed_std_all = []
missings_all = []
X_all = []
y_all = []
for fname in os.listdir('filipe_places'):
    print "reading data for place id:", fname[-6:-4]
    X, y, removed_seasonality, removed_std, missings = load_data('filipe_places/'+fname, NUM_LAGS, bootstrap_size)
    ids.append(fname[-6:-4])
    X_all.append(X)
    y_all.append(y)
    removed_seasonality_all.append(removed_seasonality)
    removed_std_all.append(removed_std)
    missings_all.append(missings)
X_all = np.array(X_all)
y_all = np.array(y_all)
print X_all.shape
print y_all.shape
n_places = len(ids)
n_instances = X_all.shape[1]
print "n_instances:", n_instances
print "n_places:", n_places
print "n_lags:", NUM_LAGS

reading data for place id: s8
reading data for place id: _k
reading data for place id: QE
reading data for place id: 5U
reading data for place id: LQ
reading data for place id: vU
reading data for place id: aM
reading data for place id: Gc
reading data for place id: kI
(9, 51552, 10, 1)
(9, 51552)
n_instances: 51552
n_places: 9
n_lags: 10


In [ ]:
# reshape data
X = np.swapaxes(X_all, 0, 1)
X = np.swapaxes(X, 1, 2)
X = X[:,:,:,:,np.newaxis]
print X.shape
y = np.swapaxes(y_all, 0, 1)
y = y[:,np.newaxis,:,np.newaxis,np.newaxis]
print y.shape

STEPS_AHEAD = 1
X = X[:,:NUM_LAGS-STEPS_AHEAD+1,:,:]
print X.shape

## Train/test split

In [ ]:
n_train = 12*24*89
n_test = n_train + 12*24*90
X_train = X[:n_train,:]
y_train = y[:n_train]
X_test = X[n_train:n_test,:]
y_test = y[n_train:n_test]
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

## Evaluation functions

In [ ]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [ ]:
def compute_error_filtered(trues, predicted, filt):
    trues = trues[filt]
    predicted = predicted[filt]
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [ ]:
def eval_quantiles(lower, upper, trues, preds):
    N = len(trues)
    icp = 1.0*np.sum((trues>lower) & (trues<upper)) / N
    diffs = np.maximum(0, upper-lower)
    mil = np.sum(diffs) / N
    rmil = 0.0
    for i in xrange(N):
        if trues[i] != preds[i]:
            rmil += diffs[i] / (np.abs(trues[i]-preds[i]))
    rmil = rmil / N
    clc = np.exp(-rmil*(icp-0.95))
    return icp, mil, rmil, clc

## Linear regression baseline

In [ ]:
# place id of interest
ix = 7
print ids[ix]

In [ ]:
# true values we want to predict
y_true = y_test[:,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

# naive baseline
preds_naive = y[n_train-STEPS_AHEAD:n_test-STEPS_AHEAD,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
corr, mae_naive, mse, rae, rmse_naive, r2_naive = compute_error(y_true, preds_naive)
print "NAIVE: %.3f   %.3f   %.3f" % (mae_naive,rmse_naive,r2_naive)

# linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train[:,:,ix,0,0], y_train[:,0,ix,0,0])
preds_lr = regr.predict(X_test[:,:,ix,0,0]) * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

corr, mae_lr, mse, rae, rmse_lr, r2_lr = compute_error(y_true, preds_lr)
print "LR:    %.3f   %.3f   %.3f" % (mae_lr,rmse_lr,r2_lr)

## Specify model

In [ ]:
def build_model(num_outs=1):
    model = Sequential()

    #model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))
    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                   input_shape=(None, 9, 1, 1),
                   padding='same', return_sequences=True))
    model.add(Dropout(0.2))

    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                       padding='same', return_sequences=False))
    
    #model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
    #           activation='linear',
    #           padding='same', data_format='channels_last'))

    #model.add(LSTM(layers[2], return_sequences=False))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    
    return model

## Train model and test

In [ ]:
print X_train.shape
print X_train[:,:,:,0,0].transpose([0,2,1]).shape
print y_train.shape

In [ ]:
model = build_model(num_outs=9)

# checkpoint best model
checkpoint = ModelCheckpoint("convlstm_mean.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit(
    X_train,
    y_train[:,:,:,:,0].swapaxes(1,2),
    batch_size=512,
    epochs=120,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)  

In [ ]:
# load weights
model.load_weights("convlstm_mean.best.hdf5")

preds_lstm = model.predict(X_test)
print preds_lstm.shape
preds_lstm = preds_lstm[:,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
print preds_lstm.shape

In [ ]:
# old results from above
print "NAIVE: %.3f   %.3f   %.3f" % (mae_naive,rmse_naive,r2_naive)
print "LR:    %.3f   %.3f   %.3f" % (mae_lr,rmse_lr,r2_lr)

# results for LSTM
corr, mae_lstm, mse, rae, rmse_lstm, r2_lstm = compute_error(y_true, preds_lstm)
print "LSTM:  %.3f   %.3f   %.3f" % (mae_lstm,rmse_lstm,r2_lstm)